In [40]:
import pandas as pd
import os
from itertools import chain
import altair as alt
import numpy as np
import ast

notebook_dir = os.getcwd()
results_path = os.path.normpath(os.path.join(notebook_dir, "outputs", "results.csv"))
norms_path   = os.path.normpath(os.path.join(notebook_dir, "data", "all_8bit_norms_with_dnf.csv"))

# Load CSVs
results_df = pd.read_csv(results_path)
norms_df   = pd.read_csv(norms_path, dtype={"8bit_vector": str})

# --- Helpers to flatten ---
def flatten_ebsn_to_str(ebsn):
    # If it's a string, convert it
    if isinstance(ebsn, str):
        ebsn = ast.literal_eval(ebsn)

    flat_list = list(chain.from_iterable(chain.from_iterable(ebsn)))
    return ''.join(str(int(b)) for b in flat_list)

def flatten_base_sn_to_str(base_sn):
    if isinstance(base_sn, str):
        base_sn = ast.literal_eval(base_sn)

    return ''.join(str(int(b)) for b in chain.from_iterable(base_sn))

def identify_base_norm(base_norm_str: str) -> str:
    """
    Identify the base social norm (e.g. Image Scoring, Stern Judging, etc.)
    from its 4-bit structure [[a,b], [c,d], ...] as stored in the dataframe.
    """
    try:
        norm = ast.literal_eval(base_norm_str)
    except Exception:
        return "Unknown"

    # Flatten if nested
    flat = [int(x) for pair in norm for x in pair]

    mapping = {
        (0, 0, 1, 1): "Image Scoring",
        (1, 0, 0, 1): "Stern Judging",
        (0, 0, 0, 1): "Shunning",
        (1, 0, 1, 1): "Simple Standing",
        (0, 0, 0, 0): "All Bad",
        (1, 1, 1, 1): "All Good",
    }

    return mapping.get(tuple(flat), "Unknown")


# Flatten columns in results
results_df['8bit_vector'] = results_df['eb_social_norm'].apply(flatten_ebsn_to_str)
results_df['4bit_orig']   = results_df['base_social_norm'].apply(eval).apply(flatten_base_sn_to_str)

# Merge and include DNF columns
merged_df = pd.merge(
    results_df,
    norms_df[["8bit_vector", 
              "Emotion_Leniency", "DNF", "DNF_literals"]],
    on=["8bit_vector"],
    how="left"
)

# Ensure numeric
merged_df["DNF_literals"] = pd.to_numeric(merged_df["DNF_literals"], errors="coerce")
merged_df["base_social_norm"] = merged_df["base_social_norm"].apply(identify_base_norm)

In [41]:
merged_df

,base_social_norm,eb_social_norm,Z,gens,mu,chi,eps,alpha,q,b,...,DISCRIMINATE,PARADOXICALLY_DISC,ALWAYS_DEFECT,EmotionProfile.COMPETITIVE,EmotionProfile.COOPERATIVE,8bit_vector,4bit_orig,Emotion_Leniency,DNF,DNF_literals
0,Simple Standing,"[[(0, 1), (0, 1)], [(0, 1), (0, 1)]]",50,100,1.0,0.01,0.01,0.0,0.5,5,...,0.00,0.080,0.900,0.90,0.10,01010101,1011,0.00,E,1
1,Simple Standing,"[[(0, 1), (0, 1)], [(0, 1), (0, 1)]]",50,100,1.0,0.01,0.01,0.0,0.5,5,...,0.00,0.000,1.000,0.00,1.00,01010101,1011,0.00,E,1
2,Simple Standing,"[[(0, 1), (0, 1)], [(0, 1), (0, 1)]]",50,100,1.0,0.01,0.01,0.0,0.5,5,...,0.14,0.000,0.860,0.86,0.14,01010101,1011,0.00,E,1
3,Simple Standing,"[[(0, 1), (0, 1)], [(0, 1), (0, 1)]]",50,100,1.0,0.01,0.01,0.0,0.5,5,...,0.98,0.000,0.020,0.02,0.98,01010101,1011,0.00,E,1
4,Simple Standing,"[[(0, 1), (0, 1)], [(0, 1), (0, 1)]]",50,100,1.0,0.01,0.01,0.0,0.5,5,...,0.98,0.020,0.000,0.02,0.98,01010101,1011,0.00,E,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31275,Stern Judging,"[[(1, 0), (0, 0)], [(0, 0), (1, 1)]]",40,500,1.0,0.10,0.10,0.0,1.0,5,...,0.00,0.000,0.975,0.03,0.97,10000011,1001,0.75,(A & R) | (~A & ~E & ~R),5
31276,Stern Judging,"[[(1, 0), (0, 0)], [(0, 0), (1, 1)]]",40,500,1.0,0.10,0.10,0.0,1.0,5,...,0.85,0.000,0.100,0.15,0.85,10000011,1001,0.75,(A & R) | (~A & ~E & ~R),5
31277,Stern Judging,"[[(1, 0), (0, 0)], [(0, 0), (1, 1)]]",40,500,1.0,0.10,0.10,0.0,1.0,5,...,0.00,0.925,0.000,0.07,0.93,10000011,1001,0.75,(A & R) | (~A & ~E & ~R),5
31278,Stern Judging,"[[(1, 0), (0, 0)], [(0, 0), (1, 1)]]",40,500,1.0,0.10,0.10,0.0,1.0,5,...,0.00,0.950,0.050,0.05,0.95,10000011,1001,0.75,(A & R) | (~A & ~E & ~R),5


In [42]:
import numpy as np

unique_norms = [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 1, 1], [0, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 1, 1, 0, 1, 1], [0, 0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 1, 1], [0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1], [0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 1, 1, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 1, 0, 1], [0, 0, 1, 1, 0, 1, 1, 1], [0, 0, 1, 1, 1, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 1, 1], [0, 1, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1], [0, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0, 0, 1, 1, 1], [0, 1, 0, 0, 1, 0, 0, 1], [0, 1, 0, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 1], [0, 1, 0, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 0, 1, 0, 1, 1, 0], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 1, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1], [0, 1, 1, 1, 0, 0, 1, 1], [0, 1, 1, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 0, 1, 1, 0], [0, 1, 1, 1, 0, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 1], [1, 0, 0, 1, 0, 0, 1, 1], [1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0, 1], [1, 1, 0, 1, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 1, 0, 1], [0, 1, 0, 0, 1, 0, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [1, 0, 0, 0, 0, 1, 1, 1], [1, 0, 1, 0, 0, 1, 0, 1], [1, 0, 1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 0, 0, 0, 1]]
# 1. Convert unique_norms from list of ints to list of strings
unique_norms_str = ["".join(map(str, norm)) for norm in unique_norms]
# --- 1. Filter and Aggregate ---

# Filter for the target gamma values
target_gammas = [0, 0.5, 1]
filtered_df = merged_df[merged_df['gamma_center'].isin(target_gammas)].copy()
filtered_df = filtered_df[filtered_df.Z==40]
filtered_df = filtered_df[filtered_df.q==1]
filtered_df = filtered_df[
    (filtered_df['Emotion_Leniency'] != 1) | (filtered_df['gamma_center'] == 0)
]
# Normalize the cooperation ratio from [0, 100] to [0, 1]
filtered_df['average_cooperation'] = filtered_df['average_cooperation'] / 100.0

mask = (filtered_df["8bit_vector"].isin(unique_norms_str)) | (filtered_df['gamma_center'] == 0)
filtered_df = filtered_df[mask]
# Select the base norm you want to visualize (e.g., "Stern Judging")
# You can wrap this in a loop if you want plots for all norms
target_base_norm = "Stern Judging" 
plot_data = filtered_df[filtered_df['base_social_norm'] == target_base_norm].copy()

# Aggregate: Get the mean cooperation for each EB social norm per gamma_center
# This averages across all simulation runs/replicates for that specific norm configuration
agg_df = plot_data.groupby(['eb_social_norm', 'gamma_center', 'Emotion_Leniency']).agg({
    'average_cooperation': 'mean'
}).reset_index()

# --- 2. Create the Altair Jitter Plot ---
jitter_chart = alt.Chart(agg_df).mark_circle(size=60, opacity=0.7).encode(
    x=alt.X('gamma_center:O', 
            title='Gamma Center',
            axis=alt.Axis(labelAngle=0)),
    y=alt.Y('average_cooperation:Q', 
            title='Average Cooperation',
            scale=alt.Scale(domain=[0, 1])),
    # The xOffset channel still pulls from our 'jitter' variable
    xOffset=alt.XOffset('jitter:Q', scale=alt.Scale(domain=[-2, 2])),    
    color=alt.Color('Emotion_Leniency:O'),
    tooltip=[
        alt.Tooltip('eb_social_norm', title='EB Norm'),
        alt.Tooltip('average_cooperation', title='Avg Coop', format='.3f'),
        alt.Tooltip('gamma_center', title='Gamma')
    ]
).transform_calculate(
    # Logic: if gamma_center is 0, jitter is 0. Else, random jitter.
    # 'datum.gamma_center' refers to the value in the current row.
    jitter='datum.gamma_center == 0 ? 0 : (random()-0.5)'
).properties(
    title=f"EB-Extension Performance: {target_base_norm}",
    width=400,
    height=500
)

jitter_chart.display()

alt.Chart(...)

In [43]:
agg_df

,eb_social_norm,gamma_center,Emotion_Leniency,average_cooperation
0,"[[(0, 0), (0, 0)], [(0, 0), (0, 1)]]",0.5,0.75,0.235684
1,"[[(0, 0), (0, 0)], [(0, 0), (0, 1)]]",1.0,0.75,0.701432
2,"[[(0, 0), (0, 0)], [(0, 1), (0, 0)]]",0.5,0.75,0.083626
3,"[[(0, 0), (0, 0)], [(0, 1), (0, 0)]]",1.0,0.75,0.461680
4,"[[(0, 0), (0, 0)], [(0, 1), (0, 1)]]",0.5,0.50,0.436659
...,...,...,...,...
166,"[[(1, 1), (1, 1)], [(0, 1), (1, 1)]]",1.0,0.75,0.683864
167,"[[(1, 1), (1, 1)], [(1, 0), (0, 1)]]",0.5,0.50,0.170754
168,"[[(1, 1), (1, 1)], [(1, 0), (0, 1)]]",1.0,0.50,0.656205
169,"[[(1, 1), (1, 1)], [(1, 1), (0, 1)]]",0.5,0.75,0.101254


In [44]:
unique_norms = [[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 1, 1, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 1, 1], [0, 0, 0, 1, 0, 1, 0, 0], [0, 0, 0, 1, 0, 1, 0, 1], [0, 0, 0, 1, 0, 1, 1, 1], [0, 0, 0, 1, 1, 0, 0, 1], [0, 0, 0, 1, 1, 0, 1, 1], [0, 0, 0, 1, 1, 1, 0, 0], [0, 0, 0, 1, 1, 1, 0, 1], [0, 0, 1, 0, 0, 0, 0, 1], [0, 0, 1, 0, 0, 0, 1, 1], [0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 1, 0, 0, 1, 0, 1], [0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 1, 0, 1, 0, 0, 1], [0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 1, 1, 0, 0, 0, 1], [0, 0, 1, 1, 0, 1, 0, 0], [0, 0, 1, 1, 0, 1, 0, 1], [0, 0, 1, 1, 0, 1, 1, 1], [0, 0, 1, 1, 1, 0, 0, 1], [0, 1, 0, 0, 0, 0, 0, 1], [0, 1, 0, 0, 0, 0, 1, 1], [0, 1, 0, 0, 0, 1, 0, 0], [0, 1, 0, 0, 0, 1, 0, 1], [0, 1, 0, 0, 0, 1, 1, 0], [0, 1, 0, 0, 0, 1, 1, 1], [0, 1, 0, 0, 1, 0, 0, 1], [0, 1, 0, 1, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 1], [0, 1, 0, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 1, 0, 0], [0, 1, 0, 1, 0, 1, 0, 1], [0, 1, 0, 1, 0, 1, 1, 0], [0, 1, 0, 1, 0, 1, 1, 1], [0, 1, 0, 1, 1, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 1, 1], [0, 1, 1, 0, 0, 1, 0, 0], [0, 1, 1, 0, 0, 1, 0, 1], [0, 1, 1, 0, 0, 1, 1, 0], [0, 1, 1, 0, 0, 1, 1, 1], [0, 1, 1, 1, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0, 0, 1], [0, 1, 1, 1, 0, 0, 1, 1], [0, 1, 1, 1, 0, 1, 0, 0], [0, 1, 1, 1, 0, 1, 0, 1], [0, 1, 1, 1, 0, 1, 1, 0], [0, 1, 1, 1, 0, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0, 1], [1, 0, 0, 0, 0, 0, 1, 1], [1, 0, 0, 0, 0, 1, 0, 0], [1, 0, 0, 0, 0, 1, 0, 1], [1, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 0, 0, 0, 1], [1, 0, 0, 1, 0, 0, 1, 1], [1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 0, 0, 1], [1, 0, 1, 0, 0, 0, 1, 1], [1, 0, 1, 0, 0, 1, 0, 0], [1, 0, 1, 1, 0, 0, 0, 0], [1, 0, 1, 1, 0, 0, 0, 1], [1, 0, 1, 1, 0, 0, 1, 1], [1, 1, 0, 0, 0, 0, 0, 1], [1, 1, 0, 1, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 1], [1, 1, 1, 1, 0, 0, 0, 1], [1, 1, 1, 1, 0, 1, 0, 0], [1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 0, 0, 1], [1, 1, 1, 1, 1, 1, 0, 1], [0, 0, 1, 0, 1, 1, 0, 1], [0, 1, 0, 0, 1, 0, 1, 1], [0, 1, 1, 0, 1, 0, 0, 1], [1, 0, 0, 0, 0, 1, 1, 1], [1, 0, 1, 0, 0, 1, 0, 1], [1, 0, 1, 1, 0, 1, 0, 0], [1, 1, 1, 0, 0, 0, 0, 1]]
# 1. Convert unique_norms from list of ints to list of strings
unique_norms_str = ["".join(map(str, norm)) for norm in unique_norms]

def flatten_ebsn_to_str(ebsn):
    # If it's a string, convert it
    if isinstance(ebsn, str):
        ebsn = ast.literal_eval(ebsn)

    flat_list = list(chain.from_iterable(chain.from_iterable(ebsn)))
    return ''.join(str(int(b)) for b in flat_list)


def get_high_performing_norms(df, base_norm_name, gamma, threshold=80):
    # Filter for base norm and gamma
    subset = df[
        (df['base_social_norm'] == base_norm_name) & 
        (df['gamma_center'] == gamma) &
        (df.Z == 40) &
        (df.q == 1) &
        (df['Emotion_Leniency'] != 1)
    ].copy()

    # 2. Use .isin() to filter for your specific unique norms
    # We use the '8bit_vector' column you already created in your main script
    subset = subset[subset['8bit_vector'].isin(unique_norms_str)]

    # 3. Aggregate
    top_norms = subset.groupby('8bit_vector').agg({
        'average_cooperation': 'mean',
        'ALWAYS_COOPERATE': 'mean',
        'DISCRIMINATE': 'mean',
        'PARADOXICALLY_DISC': 'mean',
        'ALWAYS_DEFECT': 'mean',
        'EmotionProfile.COMPETITIVE': 'mean',
        'EmotionProfile.COOPERATIVE': 'mean'
    }).reset_index()
    
    # 4. Filter for threshold
    high_perf_df = top_norms[top_norms['average_cooperation'] > threshold].copy()    
    return high_perf_df.sort_values(by='average_cooperation', ascending=False)

# Example usage:
elites = get_high_performing_norms(merged_df, "Stern Judging", 1)
print(f"Found {len(elites)} high-performing EB-norms.")

Found 8 high-performing EB-norms.


In [45]:
elites.T

,32,30,34,79,60,36,29,80
8bit_vector,01000101,01000011,01000111,11010001,10000011,01001011,01000001,11100001
average_cooperation,87.834442,85.517783,85.3103,84.61828,82.7375,82.06286,81.79972,81.3267
ALWAYS_COOPERATE,0.012987,0.079167,0.07,0.0605,0.070833,0.1355,0.0215,0.1265
DISCRIMINATE,0.642857,0.81375,0.6225,0.4925,0.787917,0.5075,0.88,0.364
PARADOXICALLY_DISC,0.324026,0.097917,0.2975,0.3995,0.10125,0.309,0.046,0.4175
ALWAYS_DEFECT,0.02013,0.009167,0.01,0.0475,0.04,0.048,0.0525,0.092
EmotionProfile.COMPETITIVE,0.347532,0.271,0.3748,0.4304,0.655667,0.225,0.1594,0.284
EmotionProfile.COOPERATIVE,0.652468,0.728667,0.6246,0.57,0.343667,0.7738,0.8406,0.7154


In [29]:
elites

,8bit_vector,average_cooperation,ALWAYS_COOPERATE,DISCRIMINATE,PARADOXICALLY_DISC,ALWAYS_DEFECT,EmotionProfile.COMPETITIVE,EmotionProfile.COOPERATIVE
48,01100101,89.93276,0.007,0.971,0.007,0.015,0.0284,0.9716
35,01001001,87.71868,0.017,0.931,0.006,0.046,0.0436,0.9564
80,11100001,81.87484,0.011,0.959,0.010,0.020,0.1352,0.8644
81,11110001,81.04168,0.018,0.869,0.045,0.068,0.1792,0.8204
46,01100011,80.99944,0.090,0.806,0.017,0.087,0.1220,0.8768
34,01000111,80.47312,0.124,0.788,0.015,0.073,0.1716,0.8272


In [17]:
# 1. Apply specific filters for EB-extensions
mask = (
    (merged_df['gamma_center'] > 0) & 
    (merged_df['Z'] == 40) & 
    (merged_df['q'] == 1) &
    (merged_df['Emotion_Leniency'] != 1) &
    (merged_df['8bit_vector'].isin(unique_norms_str)) &
    (merged_df['base_social_norm']=="Stern Judging")
)

eb_data = merged_df[mask].copy()

# 2. Count runs per norm per gamma
# We use 8bit_vector as the ID since it represents the unique extension
counts = eb_data.groupby(['gamma_center', '8bit_vector']).size().reset_index(name='n_samples')

# 3. Quick Check
summary = counts.groupby('gamma_center')['n_samples'].agg(['unique', 'count']).rename(
    columns={'unique': 'Sample_Sizes_Found', 'count': 'Number_of_Unique_Norms'}
)

print("--- EB Norm Sample Audit (Gamma > 0) ---")
print(summary)

# Display any norms that don't match the most common sample size
mode_size = counts['n_samples'].mode()[0]
mismatched = counts[counts['n_samples'] != mode_size]

if not mismatched.empty:
    print(f"\nFound {len(mismatched)} norms with inconsistent sample sizes (Expected {mode_size}):")
    print(mismatched)
else:
    print(f"\nAll {len(counts)} norm configurations have exactly {mode_size} samples.")

--- EB Norm Sample Audit (Gamma > 0) ---
                        Sample_Sizes_Found  Number_of_Unique_Norms
gamma_center                                                      
0.5                               [25, 24]                      83
1.0           [75, 50, 49, 77, 26, 25, 56]                      87

Found 86 norms with inconsistent sample sizes (Expected 25):
     gamma_center 8bit_vector  n_samples
17            0.5    00100011         24
33            0.5    01000110         24
83            1.0    00000001         75
84            1.0    00000100         75
85            1.0    00000101         75
..            ...         ...        ...
165           1.0    11110100         50
166           1.0    11110101         50
167           1.0    11110111         49
168           1.0    11111001         50
169           1.0    11111101         50

[86 rows x 3 columns]
